In [1]:
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Input, Lambda
# from sklearn.model_selection import train_test_split
from tensorflow.keras.optimizers import Adam


import random

print(tf.config.list_physical_devices('GPU'))
df = pd.read_csv("prep_data/image_data.csv")
df.head()

[]


,filename,scalar_x,scalar_y,s_button
0,1.jpg,334,181,0
1,2.jpg,335,214,0
2,3.jpg,336,193,0
3,4.jpg,337,204,0
4,5.jpg,340,130,0


In [12]:

def create_input():
    X = []
    y = []
    df = pd.read_csv("prep_data/image_data.csv")

    shuffled_df = df.sample(len(df))

    print(shuffled_df.head)  # Shuffled DataFrame
    s = 0
    for entry in shuffled_df.values:
        
        s += 1
        # print(s)
        entry_img = cv2.imread(f"prep_data/{entry[0]}")

        prepared_y = np.delete(entry,0)

        X.append(entry_img)
        y.append(prepared_y)

    y = np.asarray(y).astype(np.float32)

    return X, y


In [17]:
X, y = create_input()

<bound method NDFrame.head of     filename  scalar_x  scalar_y  s_button
171  172.jpg       347       200       100
298  299.jpg       209       115       100
169  170.jpg       344       208       100
109  110.jpg       232        74         0
319  320.jpg       232       181       100
..       ...       ...       ...       ...
573  581.jpg       437       238       100
142  143.jpg       299        97         0
134  135.jpg       278       182         0
402  555.jpg       372       130         0
426  418.jpg       219       129         0

[728 rows x 4 columns]>


In [15]:
print(y)

[[407. 235.   0.]
 [427. 184.   0.]
 [305.  83.   0.]
 [385. 183.   0.]
 [301. 129.   0.]
 [233. 102.   0.]
 [372. 130.   0.]
 [266.  77.   0.]
 [285. 214.   0.]
 [264.  86.   0.]
 [235. 210.   0.]
 [264.  86.   0.]
 [282. 103.   0.]
 [274. 119.   0.]
 [176. 184.   0.]
 [329. 197.   0.]
 [372. 233.   0.]
 [471. 274.   0.]
 [380.  45.   0.]
 [285. 246.   0.]
 [342.  99.   0.]
 [413. 209.   0.]
 [320. 219.   0.]
 [364.  86.   0.]
 [361.  67.   0.]
 [350.  61.   0.]
 [336. 193.   0.]
 [336. 193.   0.]
 [251. 101.   0.]
 [386. 202.   0.]
 [372. 233.   0.]
 [205.  88.   0.]
 [287. 238.   0.]
 [241.  86.   0.]
 [208. 123.   0.]
 [334. 181.   0.]
 [363. 191.   0.]
 [199.  66.   0.]
 [253. 179.   0.]
 [363. 191.   0.]
 [278. 182.   0.]
 [185. 127.   0.]
 [380. 107.   0.]
 [232.  74.   0.]
 [207. 119.   0.]
 [320. 219.   0.]
 [507. 177.   0.]
 [405. 118.   0.]
 [353. 102.   0.]
 [220. 249.   0.]
 [430. 131.   0.]
 [278. 250.   0.]
 [380.  45.   0.]
 [506. 189.   0.]
 [372. 130.   0.]
 [306.  77

In [4]:
def compile_model():
    image_width, image_height, channels = 640, 360, 3  # Assuming RGB images

    model = Sequential()
    model.add(Input(shape=(image_height, image_width, channels)))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(3, activation='linear'))

    model.compile(loss='mean_squared_error', optimizer=Adam())

    return model



In [5]:
def train_model(model,X,y, epochs_req):

    train_split = int(len(y) * 0.8)
    X_train, X_test = np.array(X[:train_split - 1]), np.array(X[train_split:])
    y_train, y_test = y[:train_split - 1], y[train_split:]
    
    print(X_train.shape, y_train.shape)
    model.fit(X_train, y_train, epochs=(epochs_req), validation_data=(X_test, y_test))

    for i in range(10):
        tbp = np.asarray(X_test[i])
        tbp = np.expand_dims(tbp, axis=0)
        predictions = model.predict(tbp)
        print("pred", predictions, "real", y_test[i])


    return model

In [9]:
model = compile_model()
model = train_model(model, X, y, 10)
model.save('cemetery.h5')
#create first train save

(581, 360, 640, 3) (581, 3)
Epoch 1/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 19s 915ms/step - loss: 890818.0000 - val_loss: 12867.2881
Epoch 2/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 873ms/step - loss: 8025.1362 - val_loss: 5229.3994
Epoch 3/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 870ms/step - loss: 5061.9531 - val_loss: 4246.3525
Epoch 4/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 21s 1s/step - loss: 3829.7654 - val_loss: 3987.7856
Epoch 5/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 18s 928ms/step - loss: 2939.8254 - val_loss: 3895.2063
Epoch 6/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 893ms/step - loss: 2367.7375 - val_loss: 3821.8376
Epoch 7/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 909ms/step - loss: 2005.9583 - val_loss: 3874.1499
Epoch 8/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 902ms/step - loss: 1925.4502 - val_loss: 3659.4001
Epoch 9/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 18s 955ms/step - loss: 1645.1504 - val_loss: 3614.1443
Epoch 10/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17s 888ms/step - loss: 1379.2034 - val_loss: 3841.2869
Epoch 11/38
19/19 ━━━━━━━━━━━━━━━━━━━━ 17

In [18]:
# for n in range(2):
#     print(n)
model_rap = tf.keras.models.load_model('cemetery_3.h5')
    # model_rap.summary()
    
model_rap.compile(loss='mean_squared_error', optimizer=Adam())
train_model(model_rap, X, y, 15)
model_rap.save('cemetery_4.h5')

(206, 360, 640, 3) (206, 3)
Epoch 1/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 8s 879ms/step - loss: 3008.5466 - val_loss: 380.3488
Epoch 2/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 836ms/step - loss: 1322.0321 - val_loss: 1182.0865
Epoch 3/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 844ms/step - loss: 1223.0286 - val_loss: 509.4632
Epoch 4/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 860ms/step - loss: 532.7397 - val_loss: 248.4871
Epoch 5/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 848ms/step - loss: 318.0100 - val_loss: 413.4890
Epoch 6/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 854ms/step - loss: 297.9119 - val_loss: 386.1134
Epoch 7/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 840ms/step - loss: 229.9003 - val_loss: 301.9111
Epoch 8/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 843ms/step - loss: 124.4307 - val_loss: 198.2624
Epoch 9/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 851ms/step - loss: 90.6358 - val_loss: 197.2214
Epoch 10/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 832ms/step - loss: 56.1030 - val_loss: 206.4371
Epoch 11/15
7/7 ━━━━━━━━━━━━━━━━━━━━ 6s 833ms/step - loss: 38.9856 - val_loss: 206.3854
E

pred [[295.69354  250.47452   -7.289477]] real [287. 238.   0.]


In [ ]:
# from google.colab.patches import cv2_imshow
# def draw_dot(image,button, x, y, radius=20, thickness=-1, color = None):
#     if color == None:
#       color = (255, 0, 0) if button == "left" else (0, 0, 255)
#     else:
#       color = (255, 255, 0) if button == 0 else (0, 255, 255)
#     cv2.circle(image, (x, y), radius, color, thickness)
# 
# df = pd.read_csv("/content/drive/MyDrive/prep_data/image_data.csv")
# ss = 100
# for i in range(20):
#   ss = ss + i
#   df_entry = df.loc[ss]
#   print(df_entry, type(df_entry))
# 
# 
#   img = cv2.imread(f"/content/drive/MyDrive/prep_data/{ss}.jpg")
#   tbp = np.asarray(img)
#   tbp = np.expand_dims(tbp, axis=0)
#   Y = model.predict(tbp)
#   print(Y)
#   pred_x, pred_y, pred_button = Y[0][0], Y[0][1], Y[0][2]
#   pred_x, pred_y, pred_button = round(pred_x*640), round(pred_y*360), round(pred_button)
#   print(pred_x, pred_y, pred_button)
# 
#   draw_dot(img, pred_button, pred_x, pred_y, color = pred_button)
#   cv2_imshow(img)

